In [3]:
from tvb.simulator.lab import *
%pylab nbagg
import copy
import random
import copy
import pickle
import os

   INFO  log level set to INFO
Populating the interactive namespace from numpy and matplotlib


In [4]:
savepath = "data/"
runtime = 10000
len_frame = 100
dt = 1
n_frame = len_frame/dt
ic_buffer = 3000

In [21]:
a = [ 
      #control
      ('non', ),         
     
      #Random testing cases
      ('sqr', 0.50, 4, True, 2),       
      ('sqr', 1.00, 4, True, 2),        
      ('sqr', 1.50, 4, True, 2), 
    
    
      ('sqr', 0.50, 4, True, 4),         
      ('sqr', 1.00, 4, True, 4),        
      ('sqr', 1.50, 4, True, 4),                  
      
      ('sqr', 1.00, 50, True, 2),  
      ('sqr', 1.50, 50, True, 2), 
      ('sqr', 2.00, 50, True, 2),
      
      ('sin', 0.50, 50, True), 
      ('sin', 1.00, 50, True),
      ('sin', 1.50, 50, True),
      ('sin', 2.00, 50, True),

      ('sin', 0.50, 25, True), 
      ('sin', 1.00, 25, True),
      ('sin', 1.50, 25, True), 
      ('sin', 2.00, 25, True),
    
    #more testing cases
      ('sqr', 0.50, 10, True, 10),       
      ('sqr', 1.00, 10, True, 10),        
      ('sqr', 1.50, 10, True, 10), 
    
    
      ('sqr', 0.50, 2, True, 2),         
      ('sqr', 1.00, 2, True, 2),        
      ('sqr', 1.50, 2, True, 2),                  
      
      ('sqr', 1.00, 50, True, 10),  
      ('sqr', 1.50, 50, True, 10), 
      ('sqr', 2.00, 50, True, 10),
    
      ('sin', 0.10, 25, True), 
      ('sin', 0.20, 25, True),
      ('sin', 0.30, 25, True), 
      ('sin', 0.40, 25, True),
    
      ('sin', 0.10, 10, True), 
      ('sin', 0.25, 10, True),
      ('sin', 0.50, 10, True), 
      ('sin', 1.00, 10, True),
    
      ('sin', 0.10, 5, True), 
      ('sin', 0.25, 5, True),
      ('sin', 0.50, 5, True), 
      ('sin', 1.00, 5, True),
    

    
    
    ]

In [23]:
ac = copy.deepcopy(a)

existing_data = os.listdir(savepath)

amps_all = []
actions_all = []
t_all = []
d_all = []
n_trial = 0
ioff()
while (len(ac) > 2):
    n_trial += 1
    
    actions_trial = []
    amps_trial = []
    action = random.choice(ac)
    ac.remove(action)
    
    
    if (mk_filename(action) + ".p") in existing_data:
        print "Skipping simulation " + mk_filename(action) #+ str(n_trial) + " with " + action[0] + " stimulus, " + str(action[1]) + " amp, and " + str(action[2]) + " period."
    
    else:
        print "Preparing simulation " + mk_filename(action)# str(n_trial) + " with " + action[0] + " stimulus, " + str(action[1]) + " amp, and " + str(action[2]) + " period."

        t_prep, d_prep, sim_trial, stim_trial = prepare_sim()

        for i in range(runtime / len_frame):

            #print str((n_trial, i))

            if (i - 5) % 6 == 0:
                action_frame = action
            else:
                action_frame = ('non', )
            amps_frame = translate_action(action_frame)

            amps_trial = np.concatenate((amps_trial, amps_frame))
            actions_trial.append(action_frame)

            t_frame, d_frame = run_frame(sim_trial, stim_trial, amps_frame)

            if i != 0:
                t_trial = np.concatenate((t_trial, t_frame), 0)
                d_trial = np.concatenate((d_trial, d_frame), 0)
            else:
                t_trial = t_frame
                d_trial = d_frame

        amps_all.append(amps_trial)
        actions_all.append(actions_trial)
        t_all.append(t_trial)
        d_all.append(d_trial)

        plot_and_save_trial(t_trial - min(t_trial), d_trial, amps_trial, action)

Skipping simulation sin-amp100prd10cb1
Skipping simulation sin-amp40prd25cb1
Skipping simulation sin-amp200prd25cb1
Skipping simulation sqr-amp150prd50cb1pw2
Skipping simulation sin-amp50prd5cb1
Skipping simulation sqr-amp150prd4cb1pw2
Skipping simulation sin-amp20prd25cb1
Skipping simulation sqr-amp200prd50cb1pw2
Skipping simulation sqr-amp50prd2cb1pw2
Skipping simulation sin-amp25prd10cb1
Skipping simulation sqr-amp150prd50cb1pw10
Skipping simulation sqr-amp100prd50cb1pw2
Skipping simulation sin-amp10prd25cb1
Skipping simulation sin-amp10prd10cb1
Skipping simulation sqr-amp50prd4cb1pw2
Skipping simulation sin-amp100prd5cb1
Skipping simulation sin-amp10prd5cb1
Skipping simulation sin-amp150prd25cb1
Skipping simulation sqr-amp100prd10cb1pw10
Skipping simulation sqr-amp150prd4cb1pw4
Skipping simulation sin-amp30prd25cb1
Skipping simulation sin-amp50prd10cb1
Skipping simulation sqr-amp50prd4cb1pw4
Skipping simulation sin-amp25prd5cb1
Skipping simulation sin-amp50prd50cb1
Skipping simulat

In [16]:
def mk_filename(action):
    
    if action[0] == 'non':
        return 'non'
    
    filename = action[0] + "-amp" +str(int(action[1]*100)) +  "prd" + str(int(action[2])) + "cb" + str(int(action[3]))
    if action[0] == 'sqr':
        filename += 'pw' + str(int(action[4]))
    return filename

In [18]:
def plot_and_save_trial(t, d, amps, action):
    
    
    if action[0] == 'non':
        trial_title = "Control Trial with No Stimulus"
        filename = "non"
        
    else:
        trial_title = str(action[1]) + " amp, " + str(action[2]) + "ms period"
        if action[0] == 'sqr':
            trial_title = 'Square stimulus with ' + trial_title + ", " + str(action[4]) + "pulse width \n"
        elif action[0] == 'sin':
            trial_title = 'Sinusoid stimulus with ' + trial_title

        trial_title += "Charge Balancing: "
        if action[3]:
            trial_title += "ON"
        else:
            trial_title += "OFF"

        if action[0] == 'non':
            trial_title = "Control Run with No Stimulation"

        filename = action[0] + "-amp" +str(int(action[1]*100)) +  "prd" + str(int(action[2])) + "cb" + str(int(action[3]))
        if action[0] == 'sqr':
            filename += 'pw' + str(int(action[4]))

    
    
    fig, ax1 = pyplot.subplots(figsize=(10,5))
    ax2 = ax1.twinx()
    
    ax1.set_ylim(-3, 3)
    ax1.set_ylabel('LFP')
    ax2.set_ylim(-2, 2)
    ax2.set_ylabel('Stimulus')
    
    ax2.plot(amps, 'b', alpha = 0.6)
    ax1.plot(d[:,0,9,0], 'r')
    
    title(trial_title)
    
    red = matplotlib.patches.Patch(color='red', label = 'LFP')
    blue = matplotlib.patches.Patch(color='blue', label = 'Stimulus')
    legend(handles=[red,blue])
    
    fig.savefig(str(str(savepath) + str(filename)))
    savefile = open(savepath + filename + '.p','w')
    pickle.dump((t, d, amps, action), savefile)
    
    
    

In [10]:
def run_frame(sim, stim, action_amps):
    for i in range(n_frame):
       
            
            
            
        #params_dict = {'amp' : action_amps[i], 'T' : dt, 'tau' : dt, 'onset' : dt}
        stim.temporal.parameters["amp"] = action_amps[i]
        #stim.temporal.parameters["T"] = runtime
        #stim.temporal.parameters["tau"] = runtime
        
        (t, d), (te, de), (ti, di) = sim.run(simulation_length = dt)
        
        
        if i == 0:
            t_f = copy.deepcopy(t)
            d_f = copy.deepcopy(d)
            
        else: 
            t_f = np.append(t_f, t, axis = 0)
            d_f = np.append(d_f, d, axis = 0)
    
    return t_f, d_f

In [11]:
def translate_action(action):
    
    if (len(action)) > 3:
        balance_charge = action[3]
    action_amps = []
    
    if action[0] == 'sin':
        wave_amp = action[1]
        period = action[2]        
        for i in range(n_frame):
            action_amps.append( wave_amp * np.sin( (2*np.pi / period) * i ))
        if balance_charge:
            action_amps[n_frame - 1] -= sum(action_amps)
    
    elif action[0] == 'sqr':
        wave_amp = action[1]
        period = action[2]
        pulse_width = action[4]
        for i in range(n_frame):
            if (i*dt) % period  == 0:
                amplitude = wave_amp
                
            elif (i*dt - pulse_width/2) % period == 0:
                amplitude = -wave_amp
            
            elif (i*dt - pulse_width) % period == 0:
                amplitude = 0
                
            action_amps.append(amplitude)
       
    elif action[0] == 'non':
        for i in range(n_frame):
            action_amps.append(0)
    

    
    return action_amps

#Epileptor Library

In [12]:
def prepare_sim(): 
    epileptic = [9]
    almost_epileptic = []
    stimulus_target = [9]
    stimulus_amplitude = 0
    stimulus_weight = 2 #0-2 for each region
    onset = 0
    period = runtime
    pulse_width = runtime
    frame_length = len_frame
    (s, stimulus) = set_sim(epileptic, almost_epileptic, stimulus_target, stimulus_amplitude, 
                  stimulus_weight, onset, period, pulse_width, runtime)


    (t_t, t_d), (aa, aa1), (bb, bb1) = s.run(simulation_length = ic_buffer)

    t_all = copy.copy(t_t)
    d_all = copy.copy(t_d)

    return t_all, d_all, s, stimulus

In [13]:
def set_sim(epileptic, almost_epileptic, target, amp, stim_weight, 
            onset, period, pw, sim_time, K_s=-0.2, K_f=0.1, r_1=0.00015):
  
    #Set up epileptic regions
    epileptors = models.Epileptor(Ks=K_s, Kf=K_f, r=r_1)
    epileptors.x0 = np.ones((76))*-2.4 #set all brain regions non-epileptic
    epileptors.x0[almost_epileptic] = np.ones((2))*-1.9 #overwrite as epileptogenic; borderline epileptic ('close to epilepsy threshold')
    epileptors.x0[epileptic] = np.ones((3))*-1.6 #rHC and lHC are epileptic

    #brain parameters and integration method
    con = connectivity.Connectivity(load_default=True)
    coupl = coupling.Difference(a=1.)
   


    hiss = noise.Additive(nsig = np.array([0., 0., 0., 0.0003, 0.0003, 0.]))
    #heunint = integrators.HeunStochastic(dt=0.05, noise=hiss)
    heunint = integrators.HeunDeterministic(dt=0.05)

    #set up monitors (SEEG and temporal average)
    rm = region_mapping.RegionMapping(load_default=True)
    mon_tavg = monitors.TemporalAverage(period=1.)
    mon_EEG = monitors.EEG(load_default=True,
                       region_mapping=rm,
                       period=1.)
    mon_SEEG = monitors.iEEG(load_default=True,
                             region_mapping=rm,
                             period=1.,
                             ) #intracranial EEG (iEEG) is a particular/specific type of stereoEEG (SEEG)
    what_to_watch = (mon_tavg, mon_EEG, mon_SEEG)
    
    
    #set stimulus to epileptic regions
    weighting = np.ones((76))*0 #np.zeros((76))
    weighting[target] = np.array([stim_weight]) #stimulate rHC and lHC


    #set stimulation paramaeters
    eqn_t = equations.PulseTrain()               #square wave
    eqn_t.parameters["T"] = period             #period
    eqn_t.parameters["onset"] = onset
    eqn_t.parameters["tau"] = pw 
    eqn_t.parameters["amp"] = amp
    stim = patterns.StimuliRegion(temporal = eqn_t,
                          connectivity = con, 
                          weight = weighting)
    
    #Configure space and time
    stim.configure_space()
    stim.configure_time(np.arange(0., sim_time, heunint.dt))

    #Initialise a Simulator -- Model, Connectivity, Integrator, and Monitors.
    sim = simulator.Simulator(model = epileptors, 
                      connectivity = con,
                      coupling = coupl, 
                      integrator = heunint, 
                      monitors = what_to_watch,
                      stimulus = stim,
                      simulation_length = sim_time)
    sim.configure()
    #plot_pattern(stim)
    return (sim, stim)